In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import pickle


from utils.extract_features import extract_features

In [3]:
vehicles_path = ["dataset/vehicles/KITTI_extracted", "dataset/vehicles/Far", "dataset/vehicles/Left", "dataset/vehicles/MiddleClose", "dataset/vehicles/Right"]
non_vehicles_path  = ["dataset/non-vehicles/Extras", "dataset/non-vehicles/Far", "dataset/non-vehicles/Left", "dataset/non-vehicles/MiddleClose", "dataset/non-vehicles/Right"]

vehicles = []
non_vehicles = []

for path in vehicles_path:
    all_paths = glob.glob(path+'/*.png')
    vehicles.append(all_paths)
    
for path in non_vehicles_path:
    all_paths = glob.glob(path+'/*.png')
    non_vehicles.append(all_paths)

vehicles = np.hstack(vehicles)
non_vehicles = np.hstack(non_vehicles)

In [4]:
print(len(vehicles))
print(len(non_vehicles))


9391
8968


In [5]:
colorspace = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 11
pix_per_cell = 16
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

# We use to this to see how long it takes to extract the features from the dataset
t1 = time.time()

# Pass the file_vehicle list and extract the features for cars
vehicle_features = extract_features(vehicles, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
# Pass the file_notvehicle
non_vehicle_features = extract_features(non_vehicles, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t1, 2), 'Seconds to extract HOG features...')

45.66 Seconds to extract HOG features...


In [7]:
# Create an array stack of feature vectors
X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)    
#X_scaler = StandardScaler().fit(X)
#scaled_X = X_scaler.transform(X)
    
    
# Define the labels vector
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)
print('Feature vector size:', len(X_train[0]))

Feature vector size: 1188


In [8]:
from sklearn.svm import LinearSVC
# Use a linear SVC 
svc = LinearSVC()

# Check the training time for the SVC
t1 = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t1, 2), 'is the time in seconds to train')
# Check the score of the SVC
print('Test Accuracy of Support Vector Machines = ', round(svc.score(X_test, y_test), 4))

# Check the prediction time for a single sample
t1=time.time()
n_predict = 100
print('SVC prediction: ', svc.predict(X_test[0:n_predict]))
print('Correct Predictions for',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t1, 5), 'Time ins seconds to predict', n_predict,'labels')

3.93 is the time in seconds to train
Test Accuracy of Support Vector Machines =  0.979
SVC prediction:  [0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1.
 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0.]
Correct Predictions for 100 labels:  [0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1.
 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0.]
0.002 Time ins seconds to predict 100 labels


In [9]:
# save the SVC model to disk
model_file_name = 'model.p'
try:
    with open(model_file_name, 'wb') as pfile:
        pickle.dump(
            {   'svc':svc
            },
            pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save model to', model_file_name, ':', e)
    raise
print('Model saved.')

Model saved.


In [10]:
# load the model from disk
loaded_model = pickle.load(open(model_file_name, 'rb'))
result = loaded_model['svc'].score(X_test, y_test)
print(result)

0.9790305010893247
